In [33]:
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import KarateClub

In [34]:
k = KarateClub()

In [35]:
len(k)

1

In [36]:
k.num_classes

2

In [37]:
k.num_node_features

34

In [38]:
for j,i in k[0]:
    print('{} this is the key{}'.format(j,i))

edge_index this is the keytensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
          3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 13,
         13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21,
         21, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
         27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 31,
         31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
         33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33],
        [ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  0,  2,
          3,  7, 13, 17, 19, 21, 30,  0,  1,  3,  7,  8,  9, 13, 27, 28, 32,  0,
          1,  2,  7, 12, 13,  0,  6, 10,  0,  6, 10, 16,  0,  4,  5, 16,  0,  1,
          2,  3,  0,  2, 

In [39]:
import torch_geometric

In [40]:
k[0]

Data(edge_index=[2, 156], x=[34, 34], y=[34])

In [41]:
k[0].num_nodes

34

In [42]:
kar = k[0]

In [43]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [44]:
class Graph(torch.nn.Module):
    def __init__(self):
        super(Graph,self).__init__()
        self.conv1 = GCNConv(k.num_node_features,16)
        self.conv2 = GCNConv(16,k.num_classes)
        
    def forward(self,data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x,edge_index)
        
        return F.log_softmax(x, dim=1)

In [45]:
model = Graph()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

epoch = 50
model.train()

for i in range(epoch):
    optimizer.zero_grad()
    out = model(kar)
    loss  = F.nll_loss(out,kar.y)
    print('{} its the loss'.format(loss))
    loss.backward()
    optimizer.step()

0.6940446496009827 its the loss
0.6880894899368286 its the loss
0.6802305579185486 its the loss
0.6558499932289124 its the loss
0.646148145198822 its the loss
0.6368326544761658 its the loss
0.6252492070198059 its the loss
0.6071727871894836 its the loss
0.5944912433624268 its the loss
0.5779330134391785 its the loss
0.5740532279014587 its the loss
0.5768234729766846 its the loss
0.5490034818649292 its the loss
0.5253313779830933 its the loss
0.49763381481170654 its the loss
0.46051380038261414 its the loss
0.4018058776855469 its the loss
0.41307657957077026 its the loss
0.42600393295288086 its the loss
0.39087095856666565 its the loss
0.3971790373325348 its the loss
0.3066137433052063 its the loss
0.2869694232940674 its the loss
0.32580506801605225 its the loss
0.27770310640335083 its the loss
0.2645488381385803 its the loss
0.27412149310112 its the loss
0.2334519922733307 its the loss
0.22693666815757751 its the loss
0.2087678611278534 its the loss
0.17209875583648682 its the loss
0.

In [46]:
model.eval()
_, pred = model(kar).max(dim=1)
correct = float(pred.eq(kar.y).sum().item())
print('Accuracy = {}'.format(correct/34))

Accuracy = 0.9705882352941176
